In [1]:
# Relevant module imports and installs
%pip install pulp pandas brotli fuzzywuzzy
import pandas as pd
import pulp as plp
import sys 
import os
from collections import defaultdict
from fuzzywuzzy import process
import json


[notice] A new release of pip is available: 23.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:

# Get the absolute path to the directory containing the Python file
module_path = os.path.abspath(os.path.join('..', '..'))

# Add the directory to sys.path
if module_path not in sys.path:
    sys.path.append(module_path)

# Import the data retrieval functions
from challenge_data import get_bootstrap_static_data, get_gameweek_data, update_with_gameweek_cost

# Get the actual GW7 data
bootstrap_data = get_bootstrap_static_data()
gw7_data = update_with_gameweek_cost(get_gameweek_data(7, bootstrap_data), '2024-25', 7)

# Add regions
def create_region_conversion(df):
    try:
        region_dic = json.load(open('region_conversion.json'))
    except FileNotFoundError:
        region_dic = {}

    # Find all regions in df not in region_dic
    missing_regions = set(df['Region']) - set(region_dic.keys())

    for region in missing_regions:
        print(region, ':', df[df['Region'] == region]['Name'].head(10))
        country = input('Enter country: ')
        continent = input('Enter continent: ')
        region_dic[region] = {'country': country, 'continent': continent}
    
    # Save the dictionary to a JSON file
    with open('region_conversion.json', 'w') as f:
        json.dump(region_dic, f, indent=4)

def add_region_data(df):
    # First, create or update the region conversion
    create_region_conversion(df)
    
    # Load the region conversion data
    with open('region_conversion.json', 'r') as f:
        region_dic = json.load(f)

    # Normalize the keys in region_dic
    region_dic = {str(k).strip().lower(): v for k, v in region_dic.items()}

    # Add Country and Continent columns
    df['Country'] = df['Region'].apply(lambda x: region_dic.get(str(x).strip().lower(), {}).get('country', ''))
    df['Continent'] = df['Region'].apply(lambda x: region_dic.get(str(x).strip().lower(), {}).get('continent', ''))

    # Remove the Region column
    df = df.drop('Region', axis=1)

    # Reorder the columns
    columns = df.columns.tolist()
    team_index = columns.index('Team')
    columns.remove('Country')
    columns.remove('Continent')
    new_order = columns[:team_index+1] + ['Country', 'Continent'] + columns[team_index+1:]
    df = df[new_order]

    return df

gw7_data = add_region_data(gw7_data)

# Reset index
gw7_data = gw7_data.reset_index(drop=True)

In [3]:
# Get the number of players and their list of ids
player_ids = gw7_data['ID'].tolist()
player_count = len(player_ids)

# Set up the problem
model = plp.LpProblem("fpl-gw7-challenge-hindsight", plp.LpMaximize)

# Define the decision variables
lineup = [
    plp.LpVariable(f"lineup_{i}", lowBound=0, upBound=1, cat="Integer")
    for i in player_ids
]

# Define captain variables
captain = [
    plp.LpVariable(f"captain_{i}", lowBound=0, upBound=1, cat="Integer")
    for i in player_ids
]

# Set the objective function (the number of points scored by the team, with captain's points doubled)
model += plp.lpSum([lineup[i] * gw7_data.loc[i, 'Points'] for i in range(player_count)]) + \
         plp.lpSum([captain[i] * gw7_data.loc[i, 'Points'] for i in range(player_count)])

# Constraints

# Total number of players = 5
model += plp.lpSum(lineup) == 5

# Exactly one captain
model += plp.lpSum(captain) == 1

# Captain must be in the lineup
for i in range(player_count):
    model += captain[i] <= lineup[i]

# Exactly 1 Goalkeeper
model += plp.lpSum([lineup[i] for i in range(player_count) if gw7_data.loc[i, 'Position'] == 'Goalkeeper']) == 1

# At least 1 Defender
model += plp.lpSum([lineup[i] for i in range(player_count) if gw7_data.loc[i, 'Position'] == 'Defender']) >= 1

# At least 1 Midfielder
model += plp.lpSum([lineup[i] for i in range(player_count) if gw7_data.loc[i, 'Position'] == 'Midfielder']) >= 1

# At least 1 Forward
model += plp.lpSum([lineup[i] for i in range(player_count) if gw7_data.loc[i, 'Position'] == 'Forward']) >= 1

# Solve the problem
model.solve(plp.PULP_CBC_CMD(msg=False))

# Function to print players by position
def print_players_by_position(players_dict):
    total_points = 0
    for position in ['Goalkeeper', 'Defender', 'Midfielder', 'Forward']:
        if position in players_dict:
            print(f"\n{position}:")
            for player in players_dict[position]:
                captain_str = " (C)" if player['Captain'] else ""
                points = player['Points'] * (2 if player['Captain'] else 1)
                print(f"  {player['Name']}{captain_str} - {player['Team']} - Points: {points}")
                total_points += points
    print(f"\nTotal Points: {total_points}")

# Print the results
print("Status:", plp.LpStatus[model.status])

print("\nOptimal Lineup:")
selected_players = defaultdict(list)
for i in range(player_count):
    if lineup[i].value() == 1:
        player = gw7_data.loc[i]
        selected_players[player['Position']].append({
            'Name': player['Name'],
            'Team': player['Team'],
            'Points': player['Points'],
            'Captain': captain[i].value() == 1
        })

print_players_by_position(selected_players)

Status: Optimal

Optimal Lineup:

Goalkeeper:
  Pickford - Everton - Points: 15

Defender:
  Justin - Leicester - Points: 10

Midfielder:
  Saka - Arsenal - Points: 16

Forward:
  Wood - Nott'm Forest - Points: 16
  Antonio (C) - West Ham - Points: 36

Total Points: 93
